In [1]:
import tempfile

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os 
#import sounddevice as sd
from scipy.io.wavfile import write

import tensorflow as tf
from tensorflow import keras


In [ ]:
SR = 44100
sd.default.samplerate = SR
#sd.default.channels = 2
#sd.default.device = 8

In [ ]:
duration = 5
myrecording = sd.rec(int(duration * SR), channels=2)
sd.wait()
write(f'output.wav', SR, myrecording)  # Save as WAV file 

In [ ]:
duration = 5
for i in range(0, 18):
    print("trying", i)
    sd.default.device = i
    myrecording = sd.rec(int(duration * SR), channels=2)
    sd.wait()
    write(f'output{i}.wav', SR, myrecording)  # Save as WAV file 

In [ ]:
myrecording
write('output.wav', SR, myrecording)  # Save as WAV file 
myrecording

In [ ]:
sd.query_devices()
sd.query_hostapis()


In [ ]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1
fs = 44100  # Record at 44100 samples per second
seconds = 5
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input_device_index=2,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk, exception_on_overflow = False)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()


In [ ]:
p.get_default_input_device_info()

In [ ]:
p.get_device_info_by_index(9)

In [2]:
#Common.py

import pathlib

import pandas as pd

DATA_DIR = pathlib.Path("data")

MID_TO_NAME_PATH = DATA_DIR / "mid_to_display_name.tsv"
GOOGLE_AUDIO_DATASETS = [DATA_DIR / x for x in ("audioset_eval_strong.tsv", "audioset_train_strong.tsv")]

AUDIO_DIR = DATA_DIR / "audio"
IMAGE_DIR = DATA_DIR / "image"
CHAINSAW_AUDIO_DIR = AUDIO_DIR / "chainsaw"
SPEC_DIR = IMAGE_DIR / "spectrogram"
MEL_SPEC_DIR = IMAGE_DIR / "melspectrogram"

NAMES = ["Wild animals", "Chainsaw", "Wind", "Rain", "Thunderstorm"]
LABEL_TO_NAME_DF = pd.read_csv(MID_TO_NAME_PATH, sep="\t", names=["label", "name"])


IMG_HEIGHT = IMG_WIDTH = 224


In [2]:
model = keras.models.load_model("melspec_model")

In [3]:
def signal_to_melspectrogram_image(signal, fname=None):
    # create melspectrogram
    fig, ax = plt.subplots(figsize=(0.72, 0.72))
    if fname is None:
        _, fname = tempfile.mkstemp(suffix=".jpg")
    librosa.display.specshow(librosa.power_to_db(librosa.feature.melspectrogram(signal), ref=np.max))
    ax.axis("off")
    fig.savefig(fname, dpi=400, bbox_inches="tight", pad_inches=0)
    fig.clear()
    return fname

In [17]:
#Audio Recorder

import pyaudio
import wave

audio=pyaudio.PyAudio()
stream=audio.open(format=pyaudio.paInt16,channels=1,rate=44100,input=True,frames_per_buffer=1024)

frames=[]

try:
    while True:
        data=stream.read(1024)
        frames.append(data)
except KeyboardInterrupt:
    pass

stream.stop_stream()
stream.close()
audio.terminate()

sound_file=wave.open("data/test.wav","wb")
sound_file.setnchannels(1)
sound_file.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
sound_file.setframerate(44100)
sound_file.writeframes(b''.join(frames))
sound_file.close()
        

In [2]:
audio = librosa.load(DATA_DIR/"test.wav", offset=2, duration=3)[0]
fname = signal_to_melspectrogram_image(audio)
fname

NameError: name 'DATA_DIR' is not defined

In [19]:
img = keras.preprocessing.image.load_img(fname, target_size=(IMG_HEIGHT, IMG_WIDTH))
# rescale to range [0, 1]
img_array = keras.preprocessing.image.img_to_array(img) / 255
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

class_names = ["Chainsaw", "Others"]
print("This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score)))

os.remove("data/test.wav")

This image most likely belongs to Others with a 73.11 percent confidence.
